# Companion guide to the Python [packaging tutorial](https://packaging.python.org/en/latest/tutorials/packaging-projects/).
This is not an [overview of packaging](https://drivendata.co/blog/python-packaging-2023), nor a [history of the tooling](https://pradyunsg.me/blog/2023/01/21/thoughts-on-python-packaging/). The intended audience is an author of a simple package who merely wants to publish it on the [package index](https://pypi.org), without being forced to make uninformed choices.

## Build backends
The crux of the [poor user experience](https://pradyunsg.me/blog/2023/01/21/thoughts-on-python-packaging/#pick-from-n-equivalent-choices-is-really-bad-ux) is [choosing a build backend](https://packaging.python.org/en/latest/tutorials/packaging-projects/#choosing-a-build-backend). The reader at this stage does not know what a "build backend" is, and moreover does not care.

The 4 backends in the tutorial are described here in their presented order. An example snippet of a `pyproject.toml` file is included, mostly assuming defaults, with a couple common options:

* [dynamic version](https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#static-vs-dynamic-metadata)
* package data for [type information](https://peps.python.org/pep-0561/#packaging-type-information)

### [hatchling](https://pypi.org/project/hatchling/)
[![image](https://img.shields.io/pypi/dm/hatchling.svg)](https://pepy.tech/project/hatchling) - 83 kB with 4 dependencies
```toml
requires = ["hatchling"]
build-backend = "hatchling.build"

[tool.hatch.build.targets.sdist]
include = ["<package>/*"]

[tool.hatch.version]
path = "<package>/__init__.py"
```
Part of - not to be confused with - the project manager [Hatch](https://hatch.pypa.io/latest/).

The source distribution section is included because by default hatchling ostensibly includes [all files that are not ignored](https://hatch.pypa.io/latest/plugins/builder/sdist/#default-file-selection). However, it only [abides by the root .gitignore](https://hatch.pypa.io/latest/config/build/#vcs). It will include virtual environments, if not [named .venv](https://hatch.pypa.io/dev/history/hatchling/#hatchling-v1.24.2).  For a project that advocates [sensible defaults](https://hatch.pypa.io/latest/why/#build-backend), this is surprising behavior and a security flaw. Even if [the issue](https://github.com/pypa/hatch/issues/304) is fixed, it will presumably include untracked files and clearly omissible directories such as `.github`.

### [setuptools](https://pypi.org/project/setuptools/)
[![image](https://img.shields.io/pypi/dm/setuptools.svg)](https://pepy.tech/project/setuptools) - 894 kB with 0 dependencies
```toml
[build-system]
requires = ["setuptools>=61.0"]
build-backend = "setuptools.build_meta"

[tool.setuptools]
packages = ["<package>"]

[tool.setuptools.dynamic]
version = {attr = "<package>.__version__"}
```
The original build tool, and previously the de facto standard. It is no longer commonly included in Python distributions, so they are all on equal footing with respect to needing installation.

Setuptools may require explicitly specifying the package, depending on what directories are present. It also includes legacy ".egg" and "setup.cfg" files, which a modern user will not be familiar with.

### [flit-core](https://pypi.org/project/flit-core/)
[![image](https://img.shields.io/pypi/dm/flit_core.svg)](https://pepy.tech/project/flit_core) - 63 kB with 0 dependencies
```toml
[build-system]
requires = ["flit-core>=3.4"]
build-backend = "flit_core.buildapi"
```
Part of the [Flit](https://flit.pypa.io/en/stable/) tool for publishing packages.

Flit automatically supports dynamic versions (and descriptions), and includes the source directory with data files in the source distribution.

### [pdm-backend](https://pypi.org/project/pdm-backend/)
[![image](https://img.shields.io/pypi/dm/pdm-backend.svg)](https://pepy.tech/project/pdm-backend) - 101 kB with 0 dependencies
```toml
requires = ["pdm-backend"]
build-backend = "pdm.backend"

[tool.pdm]
version = {source = "file", path = "<package>/__init__.py"}
```
Part of - not to be confused with - the project manager [PDM](https://pdm-project.org/).

PDM automatically includes the source and test directories, with data files, in the source distribution.

## Evaluations
### Popularity and endorsements
The popularity of Setuptools should be discounted because of its history. The popularity of Hatchling and PDM-backend is clearly influenced by their respective parent projects. PDM has significantly less downloads than the others, but they are all popular enough to expect longevity.

Setuptools, Hatch, and Flit are all under the [packaging authority umbrella](https://github.com/pypa/), though as the previously cited article points out, [PyPA affiliation does not indicate much](https://pradyunsg.me/blog/2023/01/21/thoughts-on-python-packaging/#not-a-pypa-project).

The tutorial "defaults" to Hatchling, which presumably is not intended as an endorsement, but will no doubt be interpreted as such.

### Size and dependencies
Setuptools is by far the largest; no surprise since it is much more than a build backend. Hatchling is the only one with dependencies, but the 3 modern ones seem appropriately lightweight.

### File selection
Wheels have a standard layout, but source distributions do not. Whether [sdist should include docs and tests](https://discuss.python.org/t/should-sdists-include-docs-and-tests/14578) is a matter of debate.

There was a time when open source software meant "distributed with an open source license", so the source distribution was the primary way to acquire the code. This all seems anachronistic in the age of distributed version control and public collaboration. Not to mention wheels are zip files which have the source code.

One [piece of advice](https://discuss.python.org/t/should-sdists-include-docs-and-tests/14578/4) is that the sdist should be buildable. Generated portable files could be included, thereby not needing the tools that generate them. But for a simple (read pure) Python project, that is not particularly relevant.

There is another issue with backends creating different artifacts when using their own build commands. This rundown only evaluated `python -m build`.

### Metadata
The modern 3 implicitly support data files. All 4 support dynamic versioning in some manner. Then again, maybe the `__version__` attribute is no longer the leading convention among the 7 options for [single-sourcing the version](https://packaging.python.org/en/latest/guides/single-sourcing-package-version/). Now that [importlib.metadata](https://docs.python.org/3.10/library/importlib.metadata.html) is no longer provisional, is that preferred?

## Recommendations
It would be disingenuous to not end with recommendations, since the refusal to - in a document titled [tool recommendations](https://packaging.python.org/en/latest/guides/tool-recommendations/) - is the problem. The PyPA endorses `pip`, `build`, and `twine` as standard tools, even though there are alternatives.

Author's disclosures: I am a long-time Python developer of several packages, and a couple with [extension modules](https://docs.python.org/3/extending/extending.html). I use no project management tools, and am not affiliated with any of these projects.

1. *flit-core* - No criticisms. The dynamic version and description feature are a plus; not having any flit-specific sections feels like less coupling.
1. *pdm-backend* - No criticisms. A natural choice if one wants tests in the source distribution.
1. *hatchling* - The file selection issue is significant. Users need a warning that they should include an sdist section and check their tarballs. Many are going to have unnecessarily large distributions, and someone with a local secrets directory - whether ignored or untracked - is going to have a seriously bad day.
1. *setuptools* - Perpetually handicapped by backwards compatibility. The only advantage setuptools had was being already installed. It may be time to disavow it for new projects without extension modules.

My projects currently use `setuptools` for purely historically reasons. For new projects, I would likely use `flit-core`. I may switch-over existing projects, though there is really no incentive to.

Unless a standard emerges, of course.

## Addendum
A meta case could be made for Flit(-core) as well: that its limited scope and independence from a project manager is itself an asset. Whereas choosing Hatch(ling) or PDM(-backend) feels like picking a side. Flit [can position itself](https://flit.pypa.io/en/latest/rationale.html) as the minimalist choice for those who resent having to choose.

And yet the situation is even more absurd. There has been an [under-documented default](https://pip.pypa.io/en/stable/reference/build-system/pyproject-toml/#fallback-behaviour) the entire time. The default is the legacy mode of setuptools, which differs only in its path setup. If one forgoes a dynamic `__version__` attribute - which perhaps `importlib.metadata` was intended to do - then one can have a `pyproject.toml` with no references to build backends.